In [3]:
from pyspark import SparkContext
import math

In [4]:
sc = SparkContext.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/05 09:23:15 WARN Utils: Your hostname, Cp resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/05 09:23:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/05 09:23:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
raw_data = sc.textFile("file:///home/cap/hadoop/hadoop-3.4.1/creditcard.csv")

## Preprocess data

In [6]:
# remove header and read data
header = raw_data.first()
data = raw_data.filter(lambda line: line != header)
parsedData = data.map(\
                lambda line: [float(x.strip('"')) for x in line.split(",")])

# check missing value
def has_missing(row):
    for item in row:
        if item is None:
            return True
        if isinstance(item, str) and (item == "" or item.lower() in ['na', 'null', 'nan']):
            return True
    return False
missing_rows = parsedData.filter(has_missing).count()
print(f"Rows with missing values: {missing_rows}")

# remove duplicate values
rdd_data = parsedData.map(lambda cols: (tuple(cols[:-1]), cols[-1])).distinct()
rdd_data = rdd_data.map(lambda x: list((list(*x[:-1],), x[-1])))


# prepare data in proper format
def prepare_features(fields):
    features = [1.0] + fields[:-1][0]
    label = fields[-1]              
    return (features, label)
rdd_data = rdd_data.map(prepare_features).cache()

# split to train and test set
train, test = rdd_data.randomSplit(weights=[0.8, 0.2], seed=42)

Rows with missing values: 0


## Functions for training model

In [7]:
# sigmoid function
def sigmoid(x):
    if x < -100:
        return 0.0
    elif x > 100: 
        return 1.0
    return 1.0 / (1.0 + math.exp(-x))

# dot function
def dot(features, weights):
    return sum(f * w for f, w in zip(features, weights))

# predict
def predict(features, weights):
    return sigmoid(dot(features, weights))

# sgd
def compute_gradient(features, label, weights):
    prediction = predict(features, weights)
    error = label - prediction
    gradient = [error * f for f in features]
    return gradient


# training model
def logistic_regression(data_rdd, learning_rate=0.1, iterations=100):
    # initialize weights
    num_features = len(data_rdd.first()[0])
    weights = [0.0] * num_features
    
    for i in range(iterations):
        # sgd for the dataset
        gradients = data_rdd.map(lambda x: compute_gradient(x[0], x[1], weights))
        
        # average gradient
        avg_gradient = gradients.reduce(lambda a, b: [x + y for x, y in zip(a, b)])
        avg_gradient = [g / data_rdd.count() for g in avg_gradient]
        
        # update weight
        weights = [w + learning_rate * g for w, g in zip(weights, avg_gradient)]

        
    
    return weights

## Testing model

Here I will experiment with two diffenrent learning rate (0.01 and 0.1) and 2 different epoch (1 and 20)

In [ ]:
print('Low-Level Operations')
# lr = 0.01, epoch = 1
final_weights = logistic_regression(train, learning_rate=0.01, iterations=1)

predictions_rdd = test.map(lambda x: (sigmoid(dot(final_weights, x[0])), x[1]))
binary_predictions_rdd = predictions_rdd.map(lambda x: (1 if x[0] >= 0.5 else 0, x[1]))
correct_predictions = binary_predictions_rdd.filter(lambda x: x[0] == x[1]).count()
total_samples = test.count()
accuracy = correct_predictions / total_samples
print(f"lr = 0.01, epoch = 1 --> Accuracy: {accuracy:.4f}")


# lr = 0.1, epoch = 1
final_weights = logistic_regression(train, learning_rate=0.1, iterations=1)

predictions_rdd = test.map(lambda x: (sigmoid(dot(final_weights, x[0])), x[1]))
binary_predictions_rdd = predictions_rdd.map(lambda x: (1 if x[0] >= 0.5 else 0, x[1]))
correct_predictions = binary_predictions_rdd.filter(lambda x: x[0] == x[1]).count()
total_samples = test.count()
accuracy = correct_predictions / total_samples
print(f"lr = 0.1, epoch = 1 --> Accuracy: {accuracy:.4f}")

# lr = 0.01, epoch = 20
final_weights = logistic_regression(train, learning_rate=0.01, iterations=20)

predictions_rdd = test.map(lambda x: (sigmoid(dot(final_weights, x[0])), x[1]))
binary_predictions_rdd = predictions_rdd.map(lambda x: (1 if x[0] >= 0.5 else 0, x[1]))
correct_predictions = binary_predictions_rdd.filter(lambda x: x[0] == x[1]).count()
total_samples = test.count()
accuracy = correct_predictions / total_samples
print(f"lr = 0.01, epoch = 20 --> Accuracy: {accuracy:.4f}")


# lr = 0.1, epoch = 20
final_weights = logistic_regression(train, learning_rate=0.1, iterations=20)

predictions_rdd = test.map(lambda x: (sigmoid(dot(final_weights, x[0])), x[1]))
binary_predictions_rdd = predictions_rdd.map(lambda x: (1 if x[0] >= 0.5 else 0, x[1]))
correct_predictions = binary_predictions_rdd.filter(lambda x: x[0] == x[1]).count()
total_samples = test.count()
accuracy = correct_predictions / total_samples
print(f"lr = 0.1, epoch = 20 --> Accuracy: {accuracy:.4f}")

Low-Level Operations


25/04/05 09:31:35 WARN BlockManager: Task 257 already completed, not releasing lock for rdd_8_0


lr = 0.01, epoch = 1 --> Accuracy: 0.9985


25/04/05 09:31:46 WARN BlockManager: Task 428 already completed, not releasing lock for rdd_8_0


lr = 0.1, epoch = 1 --> Accuracy: 0.9985


25/04/05 09:31:56 WARN BlockManager: Task 599 already completed, not releasing lock for rdd_8_0


lr = 0.01, epoch = 20 --> Accuracy: 0.9985


In [22]:
# Tính TP, FP, TN, FN
tp = binary_predictions_rdd.filter(lambda x: x[0] == 1 and x[1] == 1).count()
fp = binary_predictions_rdd.filter(lambda x: x[0] == 1 and x[1] == 0).count()
tn = binary_predictions_rdd.filter(lambda x: x[0] == 0 and x[1] == 0).count()
fn = binary_predictions_rdd.filter(lambda x: x[0] == 0 and x[1] == 1).count()

# Tính Precision và Recall
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"True positive: {tp}")
print(f"False positive: {fp}")

Precision: 0
Recall: 0.0
True positive: 0
False positive: 0
